In [1]:
import requests
import os
import json
import datetime as dt
from dateutil import parser

In [2]:
os.getcwd()

'/Users/raiha/Documents/EECS-6893-BigData/airline-topic-modeling'

In [3]:
ls

README.md
Untitled.ipynb
mentions_scraper.py
old/
run_Nov-29-2022-23-05-14_260907612_Sep-01-2022-05-00-00_Nov-10-2022-05-00-00/
run_Nov-29-2022-23-08-02_united_Sep-01-2022-05-00-00_Nov-10-2022-05-00-00/
run_Nov-29-2022-23-09-10_united_Sep-01-2022-05-00-00_Nov-10-2022-05-00-00/
run_Nov-29-2022-23-15-04_united_Sep-01-2022-05-00-00_Nov-10-2022-05-00-00/
run_Nov-29-2022-23-17-23_united_Sep-01-2022-05-00-00_Nov-10-2022-05-00-00/
run_Nov-29-2022-23-18-57_united_Sep-01-2022-05-00-00_Nov-10-2022-05-00-00/
run_Nov-29-2022-23-20-24_united_Sep-01-2022-05-00-00_Nov-10-2022-05-00-00/
run_Nov-29-2022-23-20-25_united_Sep-01-2022-05-00-00_Nov-10-2022-05-00-00/
run_Nov-29-2022-23-20-26_united_Sep-01-2022-05-00-00_Nov-10-2022-05-00-00/
run_Nov-29-2022-23-20-27_united_Sep-01-2022-05-00-00_Nov-10-2022-05-00-00/
run_Nov-29-2022-23-20-28_united_Sep-01-2022-05-00-00_Nov-10-2022-05-00-00/
run_Nov-29-2022-23-20-29_united_Sep-01-2022-05-00-00_Nov-10-2022-05-00-00/
run_Nov-29-2022-23-20-30_united_Sep-01-2022-05-

In [4]:
# To set your environment variables in your terminal run the following line:
# export 'BEARER_TOKEN'='<your_bearer_token>'
# bearer_token = os.environ.get("BEARER_TOKEN")
bearer_token = "AAAAAAAAAAAAAAAAAAAAAElqjwEAAAAA8PRQduIf5nDumMCOqlgqpkL9QHo%3D7d4vdi0UEJNMs9SmNH8nEQf9acD08THiUvzkJq3VfQzQJ4SSEf"


airline_user_id_config = {
    "@united": 260907612,
    "@AmericanAir": 22536055,
    "@SpiritAirlines": 21964954
}


def create_mentions_url(handle):
    """Creates URL to query user mentions based on user_id config above. """
    user_id = airline_user_id_config.get(handle)
    #user_id = 783214
    if user_id:
        print(user_id)
        return "https://api.twitter.com/2/users/{}/mentions".format(user_id)
    return "Not a valid Twitter handle for project scope."


def get_params(start_time, end_time, next_token=None):
    
    place_fields = [
        'full_name', 'id', 'country', 'country_code', 
        'name', 'place_type'
    ]
    
    tweet_fields = [
        'id', 'text', 'author_id', 'context_annotations',
        'conversation_id', 'created_at', 'entities', 
        'in_reply_to_user_id', 'lang', 'public_metrics', 
        'source', 'withheld'
    ]
    
    user_fields = [
        'verified'
    ]
    
    expansions = ['geo.place_id']
    
    # params = {
    #     "max_results": 100,
    #     'start_time': start_time,
    #     'end_time': end_time,
    #     #'expansions': ",".join(expansions),
    #     'tweet.fields': ",".join(tweet_fields),
    #     #'user.fields': ",".join(user_fields),
    #     #'place.fields': ",".join(place_fields)
    # }
    
    # params = {"tweet.fields": "id,text,author_id,context_annotations,conversation_id,created_at", 
    #         "max_results": 5}
    
    params = {"tweet.fields": ",".join(tweet_fields), 
              'start_time': start_time,
              'end_time': end_time,
            "max_results": 5}
    
    if next_token:
        params['pagination_token'] = next_token
        
    return params
    
    # 'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
    
    # return {"tweet.fields": "id,text,author_id,context_annotations,conversation_id,created_at", 
    #         "max_results": 5}


def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2UserMentionsPython"
    return r


def connect_to_endpoint(url, params):
    response = requests.request("GET", url, auth=bearer_oauth, params=params)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                response.status_code, response.text
            )
        )
    return response.json()


def get_json_path_to_save(handle, start_time, end_time, page_counter):
    user_id = airline_user_id_config.get(handle)
    
    date_fmt = "%b-%d-%Y-%H-%M-%S"
    run_time_str = dt.datetime.today().strftime(date_fmt)
    start_time_str = parser.parse(start_time).strftime(date_fmt)
    end_time_str = parser.parse(end_time).strftime(date_fmt)
    
    directory_name = f"run_{run_time_str}_{handle[1:]}_{start_time_str}_{end_time_str}"
    if not os.path.exists(directory_name):
        os.mkdir(directory_name)
    
    path_to_save = f"{directory_name}/{handle[1:]}_{page_counter}.json"
    return path_to_save


def save_json_to_dir(json_response, path_to_save_json):
    with open(path_to_save_json, 'w') as outfile:
        json.dump(json_response, outfile)


def main():
    url = create_mentions_url()
    params = get_params()
    json_response = connect_to_endpoint(url, params)
    print(json.dumps(json_response, indent=4, sort_keys=True))

In [5]:
dt.datetime.now().strftime('%Y-%m-%dT%H:%M:%SZ')

'2022-11-29T23:44:29Z'

In [6]:
# Test for a single day's worth of tweets: October 31st, 2022

handle = "@united"
start_time = "2022-10-31T00:00:00Z"  # sep 01 2022
end_time = "2022-11-01T00:00:00Z"  # nov 10 2022
url = create_mentions_url(handle)

page_counter = 0

params = get_params(start_time, end_time)
json_response = connect_to_endpoint(url, params)
path_to_save_json = get_json_path_to_save(handle, start_time, end_time, page_counter)
save_json_to_dir(json_response, path_to_save_json)

next_tok = json_response['meta'].get('next_token')

# while next_tok:
#     page_counter += 1
#     params = get_params(start_time, end_time, next_token=next_tok)
    
#     # Send request for next page of results
#     json_response = connect_to_endpoint(url, params)
#     path_to_save_json = get_json_path_to_save(handle, start_time, end_time, page_counter)
#     save_json_to_dir(json_response, path_to_save_json)
#     next_tok = json_response['meta'].get('next_token')

260907612
200


In [6]:
params

{'max_results': 100,
 'start_time': '2022-09-01T05:00:00.000Z',
 'end_time': '2022-11-10T05:00:00.000Z',
 'expansions': 'geo.place_id',
 'tweet.fields': 'id,text,author_id,context_annotations,conversation_id,created_at,entities,in_reply_to_user_id,lang,public_metrics,source,withheld',
 'user.fields': 'verified',
 'place.fields': 'full_name,id,country,country_code,name,place_type'}

In [6]:
print(json.dumps(json_response, indent=4, sort_keys=True))

{
    "meta": {
        "result_count": 0
    }
}


In [11]:
json_response

{'meta': {'result_count': 0}}

In [ ]:
for tweet in json_response['data'][0]:
    row_to_add = {
        "tweet_id": tweet["id"], 
        "created_at": tweet["created_at"], 
        "text": tweet["created_at"],
        
    }

In [31]:
json_response['data'][0]

{'id': '1597449399495708673',
 'edit_history_tweet_ids': ['1597449399495708673'],
 'created_at': '2022-11-29T04:36:30.000Z',
 'text': '@T_AfrikanPrince @Delta @united @JetBlue @soulplane Not soul plane tho 😂'}

In [7]:
rsp = {
    "data": [
        {
            "author_id": "3015883705",
            "context_annotations": [
                {
                    "domain": {
                        "description": "Top level entities that describe a Brands industry",
                        "id": "45",
                        "name": "Brand Vertical"
                    },
                    "entity": {
                        "id": "781974597302226944",
                        "name": "Transportation"
                    }
                },
                {
                    "domain": {
                        "description": "Entity Service top level domain, every item that is in Entity Service should be in this domain",
                        "id": "30",
                        "name": "Entities [Entity Service]"
                    },
                    "entity": {
                        "id": "781974596144668673",
                        "name": "Airline - Transportation"
                    }
                },
                {
                    "domain": {
                        "description": "Categories within Brand Verticals that narrow down the scope of Brands",
                        "id": "46",
                        "name": "Business Taxonomy"
                    },
                    "entity": {
                        "description": "Brands, companies, advertisers and every non-person handle with the profit intent related to automobile, spacecraft, aircraft and boat manufacturing",
                        "id": "1557696420500541440",
                        "name": "Automotive, Aircraft & Boat Business"
                    }
                },
                {
                    "domain": {
                        "description": "Categories within Brand Verticals that narrow down the scope of Brands",
                        "id": "46",
                        "name": "Business Taxonomy"
                    },
                    "entity": {
                        "description": "Brands, companies, advertisers and every non-person handle with the profit intent related to travel as airlines, travel agencies, hotels",
                        "id": "1557697439489880065",
                        "name": "Travel & Transportation Business"
                    }
                },
                {
                    "domain": {
                        "description": "Brands and Companies",
                        "id": "47",
                        "name": "Brand"
                    },
                    "entity": {
                        "id": "10027412598",
                        "name": "United Airlines"
                    }
                },
                {
                    "domain": {
                        "description": "Brands and Companies",
                        "id": "47",
                        "name": "Brand"
                    },
                    "entity": {
                        "id": "1582737217117560832",
                        "name": "Continental Airlines"
                    }
                },
                {
                    "domain": {
                        "description": "Entity Service top level domain, every item that is in Entity Service should be in this domain",
                        "id": "30",
                        "name": "Entities [Entity Service]"
                    },
                    "entity": {
                        "description": "Travel",
                        "id": "839159814991167489",
                        "name": "Travel"
                    }
                },
                {
                    "domain": {
                        "description": "Interests, opinions, and behaviors of individuals, groups, or cultures; like Speciality Cooking or Theme Parks",
                        "id": "67",
                        "name": "Interests and Hobbies"
                    },
                    "entity": {
                        "description": "Air travel",
                        "id": "849076200261603328",
                        "name": "Air travel"
                    }
                },
                {
                    "domain": {
                        "description": "A taxonomy of user interests. ",
                        "id": "131",
                        "name": "Unified Twitter Taxonomy"
                    },
                    "entity": {
                        "id": "10027412598",
                        "name": "United Airlines"
                    }
                },
                {
                    "domain": {
                        "description": "A taxonomy of user interests. ",
                        "id": "131",
                        "name": "Unified Twitter Taxonomy"
                    },
                    "entity": {
                        "description": "Conversation about global finance and economies.",
                        "id": "847511168268644352",
                        "name": "Global Economy"
                    }
                },
                {
                    "domain": {
                        "description": "A taxonomy of user interests. ",
                        "id": "131",
                        "name": "Unified Twitter Taxonomy"
                    },
                    "entity": {
                        "description": "Air travel",
                        "id": "849076200261603328",
                        "name": "Air travel"
                    }
                }
            ],
            "conversation_id": "1597700166777470976",
            "created_at": "2022-11-30T05:37:33.000Z",
            "edit_history_tweet_ids": [
                "1597827151600160769"
            ],
            "entities": {
                "mentions": [
                    {
                        "end": 14,
                        "id": "899081012172300288",
                        "start": 0,
                        "username": "juncalssolano"
                    },
                    {
                        "end": 164,
                        "id": "50505691",
                        "start": 153,
                        "username": "Aeromexico"
                    },
                    {
                        "end": 177,
                        "id": "21731490",
                        "start": 165,
                        "username": "VivaAerobus"
                    },
                    {
                        "end": 189,
                        "id": "227100749",
                        "start": 178,
                        "username": "flyvolaris"
                    },
                    {
                        "end": 199,
                        "id": "73485035",
                        "start": 190,
                        "username": "interjet"
                    },
                    {
                        "end": 207,
                        "id": "260907612",
                        "start": 200,
                        "username": "united"
                    }
                ]
            },
            "geo": {
                "place_id": "7d93122509633720"
            },
            "id": "1597827151600160769",
            "in_reply_to_user_id": "899081012172300288",
            "lang": "es",
            "public_metrics": {
                "like_count": 0,
                "quote_count": 0,
                "reply_count": 0,
                "retweet_count": 0
            },
            "source": "Twitter for Android",
            "text": "@juncalssolano La mayor\u00eda del costo son impuestos  y el incremento siempre ha sido el mismo.\nHay alternativas m\u00e1s econ\u00f3micas, pueden hacer uso de ellas.\n@Aeromexico @VivaAerobus @flyvolaris @interjet @united \u00bf Que opinan de este comentario?"
        },
        {
            "author_id": "1381619277187407881",
            "context_annotations": [
                {
                    "domain": {
                        "description": "Top level entities that describe a Brands industry",
                        "id": "45",
                        "name": "Brand Vertical"
                    },
                    "entity": {
                        "id": "781974597302226944",
                        "name": "Transportation"
                    }
                },
                {
                    "domain": {
                        "description": "Entity Service top level domain, every item that is in Entity Service should be in this domain",
                        "id": "30",
                        "name": "Entities [Entity Service]"
                    },
                    "entity": {
                        "id": "781974596144668673",
                        "name": "Airline - Transportation"
                    }
                },
                {
                    "domain": {
                        "description": "Categories within Brand Verticals that narrow down the scope of Brands",
                        "id": "46",
                        "name": "Business Taxonomy"
                    },
                    "entity": {
                        "description": "Brands, companies, advertisers and every non-person handle with the profit intent related to automobile, spacecraft, aircraft and boat manufacturing",
                        "id": "1557696420500541440",
                        "name": "Automotive, Aircraft & Boat Business"
                    }
                },
                {
                    "domain": {
                        "description": "Categories within Brand Verticals that narrow down the scope of Brands",
                        "id": "46",
                        "name": "Business Taxonomy"
                    },
                    "entity": {
                        "description": "Brands, companies, advertisers and every non-person handle with the profit intent related to travel as airlines, travel agencies, hotels",
                        "id": "1557697439489880065",
                        "name": "Travel & Transportation Business"
                    }
                },
                {
                    "domain": {
                        "description": "Brands and Companies",
                        "id": "47",
                        "name": "Brand"
                    },
                    "entity": {
                        "id": "10027412598",
                        "name": "United Airlines"
                    }
                },
                {
                    "domain": {
                        "description": "Brands and Companies",
                        "id": "47",
                        "name": "Brand"
                    },
                    "entity": {
                        "id": "1582737217117560832",
                        "name": "Continental Airlines"
                    }
                },
                {
                    "domain": {
                        "description": "Entity Service top level domain, every item that is in Entity Service should be in this domain",
                        "id": "30",
                        "name": "Entities [Entity Service]"
                    },
                    "entity": {
                        "description": "Travel",
                        "id": "839159814991167489",
                        "name": "Travel"
                    }
                },
                {
                    "domain": {
                        "description": "Interests, opinions, and behaviors of individuals, groups, or cultures; like Speciality Cooking or Theme Parks",
                        "id": "67",
                        "name": "Interests and Hobbies"
                    },
                    "entity": {
                        "description": "Air travel",
                        "id": "849076200261603328",
                        "name": "Air travel"
                    }
                },
                {
                    "domain": {
                        "description": "A taxonomy of user interests. ",
                        "id": "131",
                        "name": "Unified Twitter Taxonomy"
                    },
                    "entity": {
                        "id": "10027412598",
                        "name": "United Airlines"
                    }
                },
                {
                    "domain": {
                        "description": "A taxonomy of user interests. ",
                        "id": "131",
                        "name": "Unified Twitter Taxonomy"
                    },
                    "entity": {
                        "description": "Air travel",
                        "id": "849076200261603328",
                        "name": "Air travel"
                    }
                }
            ],
            "conversation_id": "1597826808715497472",
            "created_at": "2022-11-30T05:36:12.000Z",
            "edit_history_tweet_ids": [
                "1597826808715497472"
            ],
            "entities": {
                "hashtags": [
                    {
                        "end": 32,
                        "start": 20,
                        "tag": "TailTuesday"
                    },
                    {
                        "end": 48,
                        "start": 34,
                        "tag": "TwoferTuesday"
                    },
                    {
                        "end": 62,
                        "start": 50,
                        "tag": "beingunited"
                    },
                    {
                        "end": 80,
                        "start": 64,
                        "tag": "myunitedjourney"
                    }
                ],
                "mentions": [
                    {
                        "end": 89,
                        "id": "260907612",
                        "start": 82,
                        "username": "united"
                    },
                    {
                        "end": 102,
                        "id": "59256282",
                        "start": 91,
                        "username": "DENAirport"
                    }
                ],
                "urls": [
                    {
                        "display_url": "pic.twitter.com/pUjiS5LGQY",
                        "end": 126,
                        "expanded_url": "https://twitter.com/mca_photography/status/1597826808715497472/photo/1",
                        "media_key": "3_1597826806093991937",
                        "start": 103,
                        "url": "https://t.co/pUjiS5LGQY"
                    }
                ]
            },
            "id": "1597826808715497472",
            "lang": "en",
            "public_metrics": {
                "like_count": 0,
                "quote_count": 0,
                "reply_count": 0,
                "retweet_count": 0
            },
            "source": "Twitter for Android",
            "text": "A tale of two tails\n#TailTuesday \n#TwoferTuesday \n#beingunited \n#myunitedjourney \n@united \n@DENAirport https://t.co/pUjiS5LGQY"
        },
        {
            "author_id": "39559926",
            "context_annotations": [
                {
                    "domain": {
                        "description": "Top level entities that describe a Brands industry",
                        "id": "45",
                        "name": "Brand Vertical"
                    },
                    "entity": {
                        "id": "781974597302226944",
                        "name": "Transportation"
                    }
                },
                {
                    "domain": {
                        "description": "Entity Service top level domain, every item that is in Entity Service should be in this domain",
                        "id": "30",
                        "name": "Entities [Entity Service]"
                    },
                    "entity": {
                        "id": "781974596144668673",
                        "name": "Airline - Transportation"
                    }
                },
                {
                    "domain": {
                        "description": "Categories within Brand Verticals that narrow down the scope of Brands",
                        "id": "46",
                        "name": "Business Taxonomy"
                    },
                    "entity": {
                        "description": "Brands, companies, advertisers and every non-person handle with the profit intent related to automobile, spacecraft, aircraft and boat manufacturing",
                        "id": "1557696420500541440",
                        "name": "Automotive, Aircraft & Boat Business"
                    }
                },
                {
                    "domain": {
                        "description": "Categories within Brand Verticals that narrow down the scope of Brands",
                        "id": "46",
                        "name": "Business Taxonomy"
                    },
                    "entity": {
                        "description": "Brands, companies, advertisers and every non-person handle with the profit intent related to Banks, Credit cards, Insurance, Investments, Stocks ",
                        "id": "1557696848252391426",
                        "name": "Financial Services Business"
                    }
                },
                {
                    "domain": {
                        "description": "Categories within Brand Verticals that narrow down the scope of Brands",
                        "id": "46",
                        "name": "Business Taxonomy"
                    },
                    "entity": {
                        "description": "Brands, companies, advertisers and every non-person handle with the profit intent related to travel as airlines, travel agencies, hotels",
                        "id": "1557697439489880065",
                        "name": "Travel & Transportation Business"
                    }
                },
                {
                    "domain": {
                        "description": "Brands and Companies",
                        "id": "47",
                        "name": "Brand"
                    },
                    "entity": {
                        "id": "10027412598",
                        "name": "United Airlines"
                    }
                },
                {
                    "domain": {
                        "description": "Brands and Companies",
                        "id": "47",
                        "name": "Brand"
                    },
                    "entity": {
                        "id": "10043043119",
                        "name": "Boeing"
                    }
                },
                {
                    "domain": {
                        "description": "Brands and Companies",
                        "id": "47",
                        "name": "Brand"
                    },
                    "entity": {
                        "id": "1582737217117560832",
                        "name": "Continental Airlines"
                    }
                },
                {
                    "domain": {
                        "description": "Top level interests and hobbies groupings, like Food or Travel",
                        "id": "65",
                        "name": "Interests and Hobbies Vertical"
                    },
                    "entity": {
                        "id": "781974596148793345",
                        "name": "Business & finance"
                    }
                },
                {
                    "domain": {
                        "description": "Entity Service top level domain, every item that is in Entity Service should be in this domain",
                        "id": "30",
                        "name": "Entities [Entity Service]"
                    },
                    "entity": {
                        "description": "Travel",
                        "id": "839159814991167489",
                        "name": "Travel"
                    }
                },
                {
                    "domain": {
                        "description": "Top level interests and hobbies groupings, like Food or Travel",
                        "id": "65",
                        "name": "Interests and Hobbies Vertical"
                    },
                    "entity": {
                        "id": "781974596148793345",
                        "name": "Business & finance"
                    }
                },
                {
                    "domain": {
                        "description": "A grouping of interests and hobbies entities, like Novelty Food or Destinations",
                        "id": "66",
                        "name": "Interests and Hobbies Category"
                    },
                    "entity": {
                        "description": "Government",
                        "id": "857878257630195712",
                        "name": "Government"
                    }
                },
                {
                    "domain": {
                        "description": "Interests, opinions, and behaviors of individuals, groups, or cultures; like Speciality Cooking or Theme Parks",
                        "id": "67",
                        "name": "Interests and Hobbies"
                    },
                    "entity": {
                        "description": "Air travel",
                        "id": "849076200261603328",
                        "name": "Air travel"
                    }
                },
                {
                    "domain": {
                        "description": "A taxonomy of user interests. ",
                        "id": "131",
                        "name": "Unified Twitter Taxonomy"
                    },
                    "entity": {
                        "id": "10027412598",
                        "name": "United Airlines"
                    }
                },
                {
                    "domain": {
                        "description": "A taxonomy of user interests. ",
                        "id": "131",
                        "name": "Unified Twitter Taxonomy"
                    },
                    "entity": {
                        "description": "Air travel",
                        "id": "849076200261603328",
                        "name": "Air travel"
                    }
                }
            ],
            "conversation_id": "1597824396487315456",
            "created_at": "2022-11-30T05:26:36.000Z",
            "edit_history_tweet_ids": [
                "1597824396487315456"
            ],
            "entities": {
                "hashtags": [
                    {
                        "end": 17,
                        "start": 10,
                        "tag": "Boeing"
                    }
                ],
                "mentions": [
                    {
                        "end": 9,
                        "id": "260907612",
                        "start": 2,
                        "username": "united"
                    },
                    {
                        "end": 45,
                        "id": "415683367",
                        "start": 35,
                        "username": "fly2ohare"
                    }
                ],
                "urls": [
                    {
                        "description": "Photo taken at Chicago - O'Hare International (Orchard Field) (ORD / KORD) in Illinois, USA on July 10, 2022.",
                        "display_url": "airliners.net/photo/United-A\u2026",
                        "end": 95,
                        "expanded_url": "https://www.airliners.net/photo/United-Airlines/Boeing-787-10-Dreamliner/7055657",
                        "images": [
                            {
                                "height": 812,
                                "url": "https://pbs.twimg.com/news_img/1597824398085353472/3CbixY-j?format=jpg&name=orig",
                                "width": 1200
                            },
                            {
                                "height": 150,
                                "url": "https://pbs.twimg.com/news_img/1597824398085353472/3CbixY-j?format=jpg&name=150x150",
                                "width": 150
                            }
                        ],
                        "start": 72,
                        "status": 200,
                        "title": "Aviation Photo #7055657: Boeing 787-10 Dreamliner - United Airlines",
                        "unwound_url": "https://www.airliners.net/photo/United-Airlines/Boeing-787-10-Dreamliner/7055657",
                        "url": "https://t.co/8MoY8Kvtmx"
                    }
                ]
            },
            "id": "1597824396487315456",
            "lang": "en",
            "public_metrics": {
                "like_count": 0,
                "quote_count": 0,
                "reply_count": 0,
                "retweet_count": 0
            },
            "source": "Twitter Web App",
            "text": "A @united #Boeing 787-10 departing @fly2ohare Photo by Matt Kostelnick  https://t.co/8MoY8Kvtmx"
        },
        {
            "author_id": "463680615",
            "context_annotations": [
                {
                    "domain": {
                        "description": "Top level entities that describe a Brands industry",
                        "id": "45",
                        "name": "Brand Vertical"
                    },
                    "entity": {
                        "id": "781974597302226944",
                        "name": "Transportation"
                    }
                },
                {
                    "domain": {
                        "description": "Entity Service top level domain, every item that is in Entity Service should be in this domain",
                        "id": "30",
                        "name": "Entities [Entity Service]"
                    },
                    "entity": {
                        "id": "781974596144668673",
                        "name": "Airline - Transportation"
                    }
                },
                {
                    "domain": {
                        "description": "Categories within Brand Verticals that narrow down the scope of Brands",
                        "id": "46",
                        "name": "Business Taxonomy"
                    },
                    "entity": {
                        "description": "Brands, companies, advertisers and every non-person handle with the profit intent related to automobile, spacecraft, aircraft and boat manufacturing",
                        "id": "1557696420500541440",
                        "name": "Automotive, Aircraft & Boat Business"
                    }
                },
                {
                    "domain": {
                        "description": "Categories within Brand Verticals that narrow down the scope of Brands",
                        "id": "46",
                        "name": "Business Taxonomy"
                    },
                    "entity": {
                        "description": "Brands, companies, advertisers and every non-person handle with the profit intent related to travel as airlines, travel agencies, hotels",
                        "id": "1557697439489880065",
                        "name": "Travel & Transportation Business"
                    }
                },
                {
                    "domain": {
                        "description": "Brands and Companies",
                        "id": "47",
                        "name": "Brand"
                    },
                    "entity": {
                        "id": "10027412598",
                        "name": "United Airlines"
                    }
                },
                {
                    "domain": {
                        "description": "Brands and Companies",
                        "id": "47",
                        "name": "Brand"
                    },
                    "entity": {
                        "id": "1582737217117560832",
                        "name": "Continental Airlines"
                    }
                },
                {
                    "domain": {
                        "description": "Entity Service top level domain, every item that is in Entity Service should be in this domain",
                        "id": "30",
                        "name": "Entities [Entity Service]"
                    },
                    "entity": {
                        "description": "Travel",
                        "id": "839159814991167489",
                        "name": "Travel"
                    }
                },
                {
                    "domain": {
                        "description": "Interests, opinions, and behaviors of individuals, groups, or cultures; like Speciality Cooking or Theme Parks",
                        "id": "67",
                        "name": "Interests and Hobbies"
                    },
                    "entity": {
                        "description": "Air travel",
                        "id": "849076200261603328",
                        "name": "Air travel"
                    }
                },
                {
                    "domain": {
                        "description": "A taxonomy of user interests. ",
                        "id": "131",
                        "name": "Unified Twitter Taxonomy"
                    },
                    "entity": {
                        "id": "10027412598",
                        "name": "United Airlines"
                    }
                },
                {
                    "domain": {
                        "description": "A taxonomy of user interests. ",
                        "id": "131",
                        "name": "Unified Twitter Taxonomy"
                    },
                    "entity": {
                        "description": "Air travel",
                        "id": "849076200261603328",
                        "name": "Air travel"
                    }
                }
            ],
            "conversation_id": "1596587314113048576",
            "created_at": "2022-11-30T05:22:46.000Z",
            "edit_history_tweet_ids": [
                "1597823430359158784"
            ],
            "entities": {
                "mentions": [
                    {
                        "end": 11,
                        "id": "702615942157766656",
                        "start": 0,
                        "username": "fereinferi"
                    },
                    {
                        "end": 28,
                        "id": "1115334991494430720",
                        "start": 12,
                        "username": "TheRodneyOliver"
                    },
                    {
                        "end": 39,
                        "id": "425991990",
                        "start": 29,
                        "username": "nleimbach"
                    },
                    {
                        "end": 47,
                        "id": "260907612",
                        "start": 40,
                        "username": "united"
                    },
                    {
                        "end": 59,
                        "id": "702615942157766656",
                        "start": 48,
                        "username": "fereinferi"
                    }
                ]
            },
            "geo": {
                "place_id": "7142eb97ae21e839"
            },
            "id": "1597823430359158784",
            "in_reply_to_user_id": "702615942157766656",
            "lang": "en",
            "public_metrics": {
                "like_count": 0,
                "quote_count": 0,
                "reply_count": 0,
                "retweet_count": 0
            },
            "source": "Twitter for Android",
            "text": "@fereinferi @TheRodneyOliver @nleimbach @united @fereinferi ....your pic has been confiscated for further use, no follow up action is required. \n\nGood day!\ud83d\udc4d\ud83c\udffe\ud83d\ude05"
        },
        {
            "author_id": "1609491",
            "context_annotations": [
                {
                    "domain": {
                        "description": "Real world events. ",
                        "id": "29",
                        "name": "Events [Entity Service]"
                    },
                    "entity": {
                        "description": "This entity includes all conversations for all years for Thanksgiving Day ",
                        "id": "1310591085282910210",
                        "name": "Thanksgiving Day "
                    }
                },
                {
                    "domain": {
                        "description": "Top level entities that describe a Brands industry",
                        "id": "45",
                        "name": "Brand Vertical"
                    },
                    "entity": {
                        "id": "781974597302226944",
                        "name": "Transportation"
                    }
                },
                {
                    "domain": {
                        "description": "Entity Service top level domain, every item that is in Entity Service should be in this domain",
                        "id": "30",
                        "name": "Entities [Entity Service]"
                    },
                    "entity": {
                        "id": "781974596144668673",
                        "name": "Airline - Transportation"
                    }
                },
                {
                    "domain": {
                        "description": "Categories within Brand Verticals that narrow down the scope of Brands",
                        "id": "46",
                        "name": "Business Taxonomy"
                    },
                    "entity": {
                        "description": "Brands, companies, advertisers and every non-person handle with the profit intent related to automobile, spacecraft, aircraft and boat manufacturing",
                        "id": "1557696420500541440",
                        "name": "Automotive, Aircraft & Boat Business"
                    }
                },
                {
                    "domain": {
                        "description": "Categories within Brand Verticals that narrow down the scope of Brands",
                        "id": "46",
                        "name": "Business Taxonomy"
                    },
                    "entity": {
                        "description": "Brands, companies, advertisers and every non-person handle with the profit intent related to travel as airlines, travel agencies, hotels",
                        "id": "1557697439489880065",
                        "name": "Travel & Transportation Business"
                    }
                },
                {
                    "domain": {
                        "description": "Brands and Companies",
                        "id": "47",
                        "name": "Brand"
                    },
                    "entity": {
                        "id": "10027412598",
                        "name": "United Airlines"
                    }
                },
                {
                    "domain": {
                        "description": "Brands and Companies",
                        "id": "47",
                        "name": "Brand"
                    },
                    "entity": {
                        "id": "1582737217117560832",
                        "name": "Continental Airlines"
                    }
                },
                {
                    "domain": {
                        "description": "Entity Service top level domain, every item that is in Entity Service should be in this domain",
                        "id": "30",
                        "name": "Entities [Entity Service]"
                    },
                    "entity": {
                        "description": "Travel",
                        "id": "839159814991167489",
                        "name": "Travel"
                    }
                },
                {
                    "domain": {
                        "description": "Interests, opinions, and behaviors of individuals, groups, or cultures; like Speciality Cooking or Theme Parks",
                        "id": "67",
                        "name": "Interests and Hobbies"
                    },
                    "entity": {
                        "description": "Air travel",
                        "id": "849076200261603328",
                        "name": "Air travel"
                    }
                },
                {
                    "domain": {
                        "description": "Interests, opinions, and behaviors of individuals, groups, or cultures; like Speciality Cooking or Theme Parks",
                        "id": "67",
                        "name": "Interests and Hobbies"
                    },
                    "entity": {
                        "id": "1198984790092869634",
                        "name": "Holiday Travel"
                    }
                },
                {
                    "domain": {
                        "description": "Holidays like Christmas or Halloween",
                        "id": "119",
                        "name": "Holiday"
                    },
                    "entity": {
                        "description": "This entity includes all conversations for all years for Thanksgiving Day ",
                        "id": "1310591085282910210",
                        "name": "Thanksgiving Day "
                    }
                },
                {
                    "domain": {
                        "description": "A taxonomy of user interests. ",
                        "id": "131",
                        "name": "Unified Twitter Taxonomy"
                    },
                    "entity": {
                        "id": "10027412598",
                        "name": "United Airlines"
                    }
                },
                {
                    "domain": {
                        "description": "A taxonomy of user interests. ",
                        "id": "131",
                        "name": "Unified Twitter Taxonomy"
                    },
                    "entity": {
                        "description": "Travel",
                        "id": "839159814991167489",
                        "name": "Travel"
                    }
                },
                {
                    "domain": {
                        "description": "A taxonomy of user interests. ",
                        "id": "131",
                        "name": "Unified Twitter Taxonomy"
                    },
                    "entity": {
                        "description": "Air travel",
                        "id": "849076200261603328",
                        "name": "Air travel"
                    }
                },
                {
                    "domain": {
                        "description": "A taxonomy of user interests. ",
                        "id": "131",
                        "name": "Unified Twitter Taxonomy"
                    },
                    "entity": {
                        "id": "1198984790092869634",
                        "name": "Holiday Travel"
                    }
                },
                {
                    "domain": {
                        "description": "A taxonomy of user interests. ",
                        "id": "131",
                        "name": "Unified Twitter Taxonomy"
                    },
                    "entity": {
                        "description": "This entity includes all conversations for all years for Thanksgiving Day ",
                        "id": "1310591085282910210",
                        "name": "Thanksgiving Day "
                    }
                }
            ],
            "conversation_id": "1597806901495996416",
            "created_at": "2022-11-30T05:16:19.000Z",
            "edit_history_tweet_ids": [
                "1597821805380931586"
            ],
            "entities": {
                "annotations": [
                    {
                        "end": 83,
                        "normalized_text": "CRJs",
                        "probability": 0.6207,
                        "start": 80,
                        "type": "Other"
                    }
                ],
                "mentions": [
                    {
                        "end": 8,
                        "id": "6173192",
                        "start": 0,
                        "username": "jhscott"
                    },
                    {
                        "end": 16,
                        "id": "260907612",
                        "start": 9,
                        "username": "united"
                    }
                ]
            },
            "id": "1597821805380931586",
            "in_reply_to_user_id": "6173192",
            "lang": "en",
            "public_metrics": {
                "like_count": 1,
                "quote_count": 0,
                "reply_count": 0,
                "retweet_count": 0
            },
            "source": "Twitter for iPhone",
            "text": "@jhscott @united Not a lot of options to get home for thanksgiving. I think the CRJs are just showing their age. Traveling with a small baby makes everything more challenging."
        }
    ],
    "includes": {
        "places": [
            {
                "country": "Mexico",
                "country_code": "MX",
                "full_name": "Benito Ju\u00e1rez, Distrito Federal",
                "id": "7d93122509633720",
                "name": "Benito Ju\u00e1rez",
                "place_type": "city"
            },
            {
                "country": "United States",
                "country_code": "US",
                "full_name": "Georgia, USA",
                "id": "7142eb97ae21e839",
                "name": "Georgia",
                "place_type": "admin"
            }
        ]
    },
    "meta": {
        "newest_id": "1597827151600160769",
        "next_token": "7140dibdnow9c7btw424ccx07dw1yv0k0eszt36uonkmu",
        "oldest_id": "1597821805380931586",
        "result_count": 5
    }
}

In [16]:
rsp['meta']['oldest_id']

'1597821805380931586'

In [8]:
rsp.keys()

dict_keys(['data', 'includes', 'meta'])

In [12]:
rsp['data'][0]['created_at']

'2022-11-30T05:37:33.000Z'

In [13]:
rsp['data'][0]['id']

'1597827151600160769'

In [14]:
rsp['data'][4]['created_at']

'2022-11-30T05:16:19.000Z'

In [15]:
rsp['data'][4]['id']

'1597821805380931586'